# Caso de uso para la aplicación

In [1]:
import pymongo
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, POST, JSON
import requests
import pprint
import re

d:\Proyectos\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_config = dotenv_values("../.env")

### Aux functions

In [3]:
def to_title_case(s):
    return re.sub(r"(\s+|[;,.])", "", s.title())

def to_camel_case(s):
    title_case = to_title_case(s)
    return title_case[0].lower() + title_case[1:]

def get_titulos(boe_ids):
    sparql.setQuery(PREFIXES + f"""
    SELECT ?entradaBOE ?titulo
    WHERE {{
    ?entradaBOE rdf:type :EntradaBOE .
    ?entradaBOE :identificador "{boe_id}" .
    ?entradaBOE :titulo ?titulo .
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"][0]

### Creando conexiones

In [4]:
mongo_client = pymongo.MongoClient(
    host=env_config['MONGODB_HOST'],
    port=int(env_config['MONGODB_PORT']),
    username=env_config['MONGO_USER'],
    password=env_config['MONGO_PASSWORD'],
)
mongo_collection = mongo_client["boe_db"]["boe"]

In [5]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

In [6]:
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}"
)
sparql.setMethod(POST)

## Selección del BOE(s) que aplican al tema elegido

In [7]:
tema_elegido = "situacion del puesto de trabajo número de orden 9"

### Búsqueda semántica

In [8]:
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(tema_elegido),
    "k": 10,
    "num_candidates": 10,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)

pprint.pprint(res["hits"]["hits"])

[{'_id': 'HqecyosB82svYKLnQK43',
  '_index': 'boe',
  '_score': 0.8048749,
  '_source': {'doc_id': 'BOE-A-2023-8320',
              'text': 'En la página\xa045321 se modifica el código del puesto '
                      'de trabajo número de orden\xa09, donde dice: «1200440», '
                      'debe decir: «2564575».'}},
 {'_id': 'IqeiyosB82svYKLn467e',
  '_index': 'boe',
  '_score': 0.8048749,
  '_source': {'doc_id': 'BOE-A-2023-8320',
              'text': 'En la página\xa045321 se modifica el código del puesto '
                      'de trabajo número de orden\xa09, donde dice: «1200440», '
                      'debe decir: «2564575».'}},
 {'_id': 'JqeiyosB82svYKLn5a7W',
  '_index': 'boe',
  '_score': 0.75107706,
  '_source': {'doc_id': 'BOE-A-2023-7897',
              'text': 'Una vez llevado a cabo el procedimiento establecido en '
                      'la legislación vigente respecto de los procedimientos '
                      'normales de provisión de puestos de traba

In [9]:
count = {}
for parrafo in res['hits']["hits"]:
    try:
        count[parrafo['_source']['doc_id']] += 1
    except:
        count[parrafo['_source']['doc_id']] = 1
        
print("Resultados de la búsqueda: (BOE: nº de apariciones)")
pprint.pprint(count)
boe_id = max(count, key=count.get)

boe_id = res['hits']["hits"][0]['_source']['doc_id'] # Ambos están relacionados, pero para el ejemplo cogemos BOE-A-2023-8320 ya que tiene relaciones anterior y posterior

print(f"BOE elegido: {boe_id}")

Resultados de la búsqueda: (BOE: nº de apariciones)
{'BOE-A-2023-7897': 8, 'BOE-A-2023-8320': 2}
BOE elegido: BOE-A-2023-8320


## Tratamiento del BOE objetivo en GraphDB

### Mapeando posibles relaciones

In [10]:
posteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"posteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["posteriores"]:
        relacion = anterior["relacion"]
        posteriores[relacion["codigo"]] = str(relacion["codigo"]) + to_camel_case(relacion["texto"])

anteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"anteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["anteriores"]:
        relacion = anterior["relacion"]
        anteriores[relacion["codigo"]] = str(relacion["codigo"]) + to_camel_case(relacion["texto"])

In [11]:
v_relaciones = dict()
v_relaciones['anteriores'] = list(anteriores.values())
v_relaciones['posteriores'] = list(posteriores.values())

### Consulta documento objetivo

In [12]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

#### Procesamos el documento elegido y todos sus atributos y relaciones

In [13]:
describe_query = PREFIXES + f"""
                DESCRIBE ?entradaBOE
                WHERE {{
                ?entradaBOE rdf:type :EntradaBOE .
                ?entradaBOE :identificador "{boe_id}" .
                }}
                """
sparql.setQuery(query=describe_query)
sparql.setReturnFormat(JSON)
sparql.addParameter('Content-Type', 'application/sparql-query')
results = sparql.query().convert().decode("utf-8")

d:\Proyectos\venv310\lib\site-packages\SPARQLWrapper\Wrapper.py:794: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn(
d:\Proyectos\venv310\lib\site-packages\SPARQLWrapper\Wrapper.py:1179: RuntimeWarning: Format requested was JSON, but N3 (application/n-triples;charset=UTF-8) has been returned by the endpoint
  warnings.warn(


In [14]:
found_rels = dict()
for relacion in v_relaciones['anteriores']:
    if relacion in results:
        found_rels[relacion] = 'anterior'

for relacion in v_relaciones['posteriores']:
    if relacion in results:
        found_rels[relacion] = 'posterior'

#### Obtención de BOEs relacionados

In [15]:
def get_referenced_boes(boe_id, relaciones):
    select = "SELECT ?entradaBOE " + " ".join([f"?boe_anterior_{relacion} ?boe_posterior_{relacion} " for relacion in relaciones.keys()])
    where = f"""
            WHERE {{
                ?entradaBOE rdf:type :EntradaBOE .
                ?entradaBOE :identificador "{boe_id}" .
            """
    for relacion in relaciones.keys():
        if relaciones[relacion] == 'anterior':
            get_relacion_ant = f"""
                OPTIONAL {{
                        ?entradaBOE :{relacion} ?boe_anterior_{relacion} .
                        FILTER EXISTS {{ ?entradaBOE :{relacion} ?boe_anterior_{relacion} }}
                    }}
            """
            where += get_relacion_ant
        else:
            get_relacion_post = f"""
                OPTIONAL {{
                            ?entradaBOE :{relacion} ?boe_posterior_{relacion} .
                            FILTER EXISTS {{ ?entradaBOE :{relacion} ?boe_posterior_{relacion} }}
                        }}
            """
            where += get_relacion_post
    where += "\n}"

    query_relaciones = PREFIXES + select + where
    sparql.setQuery(query_relaciones)
    sparql.setReturnFormat(JSON)
    sparql.addParameter('Content-Type', 'application/sparql-query')
    results = sparql.query().convert()
    
    anteriores = []
    posteriores = []
    headers = results["head"]["vars"]
    for result in results["results"]["bindings"]:
        entradaBOE = result["entradaBOE"]["value"].split("/")[-1]
        for header in headers:
            try:    
                if result[header]["value"].split("/")[-1] == boe_id: continue
            except:
                continue
            if header.startswith("boe_anterior"):
                anteriores.append((result[header]["value"].split("/")[-1], header.split("_")[-1]))
            else:
                posteriores.append((result[header]["value"].split("/")[-1], header.split("_")[-1]))
    
    return {'anteriores': anteriores, 'posteriores': posteriores}

#### Muestra de los BOEs relacionados

In [16]:
refs = get_referenced_boes(boe_id, found_rels)
print(f"Entrada seleccionada del BOE: {boe_id}")

print("\nBOEs anteriores:")
for ref in refs['anteriores']:
    print(f"\t*{ref[1]} de {ref[0]}: {get_titulos(ref[0])['titulo']['value']}")

print("\nBOEs posteriores:")
for ref in refs['posteriores']:
    print(f"\t*{ref[1]} en {ref[0]}: {get_titulos(ref[0])['titulo']['value']}")

Entrada seleccionada del BOE: BOE-A-2023-8320

BOEs anteriores:
	*203corrigeErrores de BOE-A-2023-7897: Resolución de 31 de marzo de 2023, de la Subsecretaría, por la que se corrigen errores de la de 23 de marzo de 2023, por la que se resuelve el concurso específico convocado por la Resolución de 20 de enero de 2023.

BOEs posteriores:
	*203seCorrigenErrores en BOE-A-2023-8500: Resolución de 31 de marzo de 2023, de la Subsecretaría, por la que se corrigen errores de la de 23 de marzo de 2023, por la que se resuelve el concurso específico convocado por la Resolución de 20 de enero de 2023.
